## Get Started with DFlight API

* Create and send HTTP requests to each of the DFlight API endpoints
* Understand your results

Please review DFlight documentation before getting started:
* [Overview](https://ljaero.com/solutions/dflight/overview/)
* [Developer Resources](https://ljaero.com/solutions/dflight/dev/)
* [OpenAPI Specification](https://dflight-api.ljaero.com/)

## Requirements

In order to execute all code in this notebook, the following packages must be installed in your environment:
* [geojson](https://github.com/jazzband/geojson#installation)
* [pyproj](https://pyproj4.github.io/pyproj/stable/installation.html)
* [requests](https://requests.readthedocs.io/en/latest/user/install/#install)
* [shapely](https://shapely.readthedocs.io/en/stable/index.html)

In [ ]:
import geojson as gj
import json
import pyproj
import requests
import shapely.geometry as shpgeo

## Enter your API Key
In order to make calls to DFlight API enpoints you need to include your API Key in the ```x-api-key``` header. Replace ```nnnn``` below with your key. If you do not already have a DFlight subscription, click [here](https://ljaero.com/solutions/dflight/subscribe/) for a free trial which will allow you to make 50 API calls.

In [ ]:
mykey = 'nnnn'
headers = {'x-api-key': mykey}

## Endpoint URLs

All DFlight API endpoints are of the form

**```https://{domain}/{coverage area}/{version}/{category}/{query type}```**

where

* domain = [dflight-api.ljaero.com](https://dflight-api.ljaero.com/)
* coverage area = us
* version = v1
* category is one of: [aerodromes](https://ljaero.com/solutions/dflight/overview/#ep_aerodromes), [airspace](https://ljaero.com/solutions/dflight/overview/#ep_airspace), [obstacles](https://ljaero.com/solutions/dflight/overview/#ep_obstacles), [restrictions](https://ljaero.com/solutions/dflight/overview/#ep_tfr), [ssa](https://ljaero.com/solutions/dflight/overview/#ep_ssa), [uoa](https://ljaero.com/solutions/dflight/overview/#ep_uoa), [venues](https://ljaero.com/solutions/dflight/overview/#ep_venues), [wx-forecast](https://ljaero.com/solutions/dflight/overview/#ep_weather)
* query type is one of: ```distance-query```, ```route-query```, ```polygon-query```


In [ ]:
# Helper method to get endpoint url for a given category and query type

def get_url(cat: str, qtype: str) -> str:
    cc = ['aerodromes', 'airspace', 'obstacles', 'restrictions', 'ssa', 'uoa', 'venues', 'wx-forecast']
    qq = ['distance', 'route', 'polygon']
    if cat in cc and qtype in qq:
        return f'https://dflight-api.ljaero.com/us/v1/{cat}/{qtype}-query'    
    return None

## Composing the query request body

DFlight API requests bodies must be valid json objects; the required properties will depend on the query type and category. The query type will dictate the property(ies) used to define your geographic area of interest, and the category will determine whether or not there are additional required properties.

#### Distance queries
> For ```/distance-query``` endpoints you define the area of interest with the following properties
>* **latitude**: WGS84 latitude coordinate of your selected point, in decimal degrees
>* **longitude**: WGS84 longitude coordinate of your selected point, in decimal degrees
>* **distance**: radial distance in meters (max allowed value is 25000 m)
>
> Example:
>
>```
>{
>  "longitude": -89.419,
>  "latitude": 43.0472,
>  "distance": 5000
>}
>```

#### Route queries
> For ```/route-query``` endpoints you define the area of interest with a single property named **route**, which must be a valid [GeoJSON LineString](https://www.rfc-editor.org/rfc/rfc7946.html#appendix-A.2). The maximum allowed route length is 50 km.
>
> Example:
>
>```
>{
>  "route": {
>    "type": "LineString", 
>    "coordinates": [[-89.61,43.76], [-89.65,43.76], [-89.70,43.75], [-89.71,43.74], [-89.79,43.74]]
>   }
>}
>```

#### Polygon queries
> For ```/polygon-query``` endpoints you define the area of interest with a single property named **poly**, which must be a valid [GeoJSON Polygon](https://www.rfc-editor.org/rfc/rfc7946.html#appendix-A.3). The maximum allowed polygonal area is 1000 km<sup>2</sup>.
>
> Example:
>
>```
>{
>  "poly": {
>    "type": "Polygon",
>    "coordinates": [[[-81.425,28.333],[-81.450,28.199],[-81.133,28.176],[-81.125,28.349],[-81.425,28.333]]]
>  }
>}
>```

#### Basic Requests
> We refer to requests which only require input of your area of interest as "basic". The categories which use basic requests are:
>* aerodromes
>* obstacles
>* restrictions
>* ssa
>* uoa
>* venues
>
> When sending requests to any of the basic endpoints, the examples above represent the full request bodies... no additional properties are needed.

#### Filtered Requests
> The ```/airspace/``` and ```/wx-forecast/``` category endpoints have additional required properties in the request body json, beyond those used to define the area of interest.
>
> * For **airspace** requests, you need to include the ```asptypes``` property. The value given must be list of one or more airspace types you wish to retrieve. Allowed values are "CAS", "SUA", "MAA", and "MTR". Example:
>```
>{
>  "longitude": -88.2,
>  "latitude": 44.6,
>  "distance": 8000,
>  "asptypes": ["MAA","MTR"]
>}
>```
> * For **wx-forecast** requests, you need to include the ```wxtypes``` and ```hours``` properties. The value given for wxtypes must be a list containing one or more weather elements. Allowed values are "CIG", "DEWPT", "SKY", "TEMP", "VIS", "WINDDIR", "WINDGUST", and "WINDSPEED". The value given for hours must be an integer between 1 and 24. This will be the number of hourly forecasts to return. For the current hour only, you would enter a value of 1. To also retrieve forecasts for each of the next n hours, enter n. Example:
>```
>{
>  "longitude": -88.2,
>  "latitude": 44.6,
>  "distance": 5000,
>  "wxtypes": ["WINDDIR","WINDGUST","WINDSPEED"]
>  "hours": 4
>}
>```

In [ ]:
# Notice the spatial extent restrictions stated above for each query type. To avoid 422 errors when calling the API, you can
# use the helper methods below to ensure your GeoJSON elements are valid and within extent limits before sending requests.

def route_length_km(rte: dict) -> float:
    geod = pyproj.Geod(ellps="WGS84")
    lons, lats = zip(*rte['coordinates'])
    l = geod.line_length(list(lons), list(lats))
    return l/1000.0  # convert to km

def poly_area_km2(poly: dict) -> float:
    geod = pyproj.Geod(ellps="WGS84")
    pc = shpgeo.shape(json.loads(json.dumps(poly)))
    a = abs(geod.geometry_area_perimeter(pc)[0])
    return a/1000000.0  # convert to km^2

def is_valid_route(req_route: dict) -> bool:
    # Is this a valid GeoJSON LineString?
    if isinstance(gj.loads(json.dumps(req_route)), gj.geometry.LineString):
        # Is it less than or equal to 50 km in length?
        if route_length_km(req_route) > 50.0:
            return False
        return True
    return False

def is_valid_poly(req_polygon: dict) -> bool:
    # Is this a valid GeoJSON Polygon?
    if isinstance(gj.loads(json.dumps(req_polygon)), gj.geometry.Polygon):
        # Is the enclosed area less than or equal to 1000 km^2?
        if poly_area_km2(req_polygon) > 1000.0:
            return False
        return True
    return False

## Sending Requests

We'll use the [Requests](https://requests.readthedocs.io/) library to make our POST requests. The parameters we'll need to include when creating the requests are:
* ```url```: the endpoint url
* ```headers```: dictionary of headers to send (we created it already in code cell [2])
* ```json```: the json request body

In [ ]:
# A distance-query for Special Security Areas. The ssa endpoint is "basic"

dreq = {
  "longitude": -97.362,
  "latitude": 35.416,
  "distance": 5000
}

ssa_resp = requests.post(get_url('ssa', 'distance'), headers = headers, json = dreq)

## Working with results

```ssa_resp``` will contain a [Response](https://requests.readthedocs.io/en/latest/api/#requests.Response) object.

In [ ]:
# Check the status code of the response
ssa_resp

Status code of 200 means our results will be as described in the DFlight API specification, i.e. a json object containing a single item ```found```. The contents of ```found``` will be a GeoJSON FeatureCollection, with one Feature for each ssa found.

In [ ]:
# The json response body
ssa_json = ssa_resp.json()
ssa_json

In [ ]:
# The FeatureCollection containing ssa instances found
ssa_found = ssa_json['found']

# ssa_found is still a plain dict
print(type(ssa_found))

# In order to further work with the geometries you'll want to convert it to an actual GeoJSON FeatureCollection
ssa_fc = gj.loads(gj.dumps(ssa_found))
print(type(ssa_fc))

In [ ]:
# How many features (ssa) are there?
len(ssa_fc)

In [ ]:
feature1 = ssa_fc[0]

In [ ]:
# What is the geometry type?
type(feature1['geometry'])

In [ ]:
# How big is it (in square km)?
poly_area_km2(feature1['geometry'])

In [ ]:
# What properties are given?
feature1['properties'].keys()